<a href="https://colab.research.google.com/github/RickyDoan/DL-Tensor-Flow-Prediction/blob/main/TF_Bert_Email_Spam_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
!pip install tensorflow_text
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import tf_keras as keras

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/A Learning Tensor Flow/Bert-Email Spam/spam.csv", encoding='latin-1', usecols=['v1','v2'])
df.head()
print(df.shape)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(df[['v2']], df['v1'])

In [ ]:
y_resampled.value_counts()

In [ ]:
df_ham_sampled = df_ham.sample(df_spam.shape[0])
df_ham_sampled.shape

In [ ]:
df_sample = pd.concat([df_spam, df_ham_sampled], axis=0)
print(df_sample.shape)
df_sample.head()

In [ ]:
df_sample['v1'].value_counts()

In [ ]:
df_sample['v1'] = df_sample['v1'].map({'spam':1,'ham':0})

df_sample = df_sample.rename(columns ={
    'v2':'email',
    'v1': 'spam'
})
df_sample.head()


In [ ]:
df_sample['spam'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

X = df_sample['email']
y = df_sample['spam']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

In [ ]:
text_input = keras.layers.Input(shape=(), dtype=tf.string)
# bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
    trainable=False)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [ ]:
s = keras.layers.Dropout(0.2,name ='dropout')(pooled_output)
s = keras.layers.Dense(64,activation ='relu',name = 'h64')(s)
s = keras.layers.Dense(32,activation ='relu',name ='h34')(s)
s = keras.layers.Dense(8,activation ='relu',name ='h8')(s)
s = keras.layers.Dense(1,activation ='sigmoid',name ='output')(s)

model = keras.Model(text_input,s)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
with tf.device('/device:GPU:0'):
    model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
text = "URGENT! You have won a 1 week FREE membership in our Â£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18"

In [ ]:
prediction = model.predict([text])
prediction = np.where(prediction > 0.5, 1, 0)
if prediction[0][0] == 1:
  print("Spam")
else:
  print("Ham")

In [ ]:
df.head()

In [ ]:
df['v1'].value_counts()

In [ ]:
df['v1'] = df['v1'].map({'spam':1,'ham':0})

df = df.rename(columns ={
    'v2':'email',
    'v1': 'spam'
})
df.head()

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(df[['email']], df['spam'])

In [ ]:
y_resampled.value_counts()

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
text_input = keras.layers.Input(shape=(), dtype=tf.string)
# bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
    trainable=False)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [ ]:
s = keras.layers.Dropout(0.2,name ='dropout')(pooled_output)
s = keras.layers.Dense(64,activation ='relu',name = 'h64')(s)
s = keras.layers.Dense(32,activation ='relu',name ='h34')(s)
s = keras.layers.Dense(8,activation ='relu',name ='h8')(s)
s = keras.layers.Dense(1,activation ='sigmoid',name ='output')(s)

model2 = keras.Model(text_input,s)

In [ ]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
    model2.fit(X_train2, y_train2, epochs=10)

In [ ]:
model2.evaluate(X_test2, y_test2)

In [ ]:
y_pred2 = model2.predict(X_test2)
y_pred2 = np.where(y_pred2 > 0.5, 1, 0)
report = classification_report(y_test2, y_pred2)
print(report)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test2, y_pred2)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
text2 = "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv"

In [ ]:
prediction2 = model2.predict([text2])
prediction2 = np.where(prediction2 > 0.5, 1, 0)
if prediction2[0][0] == 1:
  print("Spam")
else:
  print("Ham")